In [1]:
from datetime import datetime, date
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF

In [5]:
#Work
uri = 'http://id.loc.gov/resources/works/1893399.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('conjecturas.nt')

<Graph identifier=N3fbb92ab2cfe46808511e64b33f28728 (<class 'rdflib.graph.Graph'>)>

In [3]:
#Instancia
uri = 'http://id.loc.gov/resources/instances/1893399.bibframe.rdf'
g = Graph()
g.parse(uri)

<Graph identifier=N674239d92a534abb9b1072b9ddf8411c (<class 'rdflib.graph.Graph'>)>

In [10]:
#Item
uri = 'http://id.loc.gov/resources/items/748300-050-10.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('item.nt')

<Graph identifier=N464b12f62d33496caea88d06e54c9b0d (<class 'rdflib.graph.Graph'>)>